In [1]:
import numpy as np
import random
from PIL import Image
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.optimizers import Adam, Nadam, Adamax
sizes = (80,80,1)

class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=50000)
        self.gamma = 0.999   # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.00  # exploration will not decay futher
        self.epsilon_decay = 0.000995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.weight_backup = 'model_weights.h5'

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=8, subsample=(4, 4), activation='relu', padding='same', input_shape= sizes))#80*80*4
        model.add(Conv2D(64, kernel_size=4, subsample=(2, 2), activation='relu', padding='same'))
        model.add(Conv2D(64, kernel_size=3, subsample=(1, 1), activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    '''
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(20, input_dim=self.state_size, activation='elu'))
        #model.add(Dropout(0.1))
        model.add(Dense(20, activation='elu'))
        #model.add(Dropout(0.2))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Nadam(lr=self.learning_rate))
        return model
    '''
    
    def save_model(self):
            self.model.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append((state, action, reward, new_state, done))

    def memory_replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        Sample = random.sample(self.memory, batch_size)
        winsample = [s for s in self.memory if s[2]== 1.0]
        #print(winsample)
        tuple(winsample)
        if len(winsample) > batch_size:
            Samplewin = random.sample(winsample, batch_size)
            Sample += Samplewin
        else:  
            Sample += winsample
        for state, action, reward, new_state, done in Sample:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(new_state))
            target_f = self.model.predict(state)
            print(action, reward, target_f)
            target_f[0] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
    '''        
    def RGBprocess(new_state):
        processed_observation = Image.fromarray(new_state, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(1, processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation
        #stack4.append(processed_observation)
        #if len(stack4) == 4:
            #stack_of_observation = np.stack((processed_observation, processed_observation, processed_observation, processed_observation), axis=2)
            #stack_of_observation = stack_of_observation.reshape(stack_of_observation.shape[0], stack_of_observation.shape[1], stack_of_observation.shape[3], stack_of_observation.shape[2])
            #print(stack_of_observation.shape)
    '''

Using TensorFlow backend.


In [2]:
import sys
import gym
from gym import wrappers

def RGBprocess(raw_img):
        processed_observation = Image.fromarray(raw_img, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(1, processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation

batch_size = 16
#episodes = sys.argv[1] if len(sys.argv) > 1 else 5000
#env_name = sys.argv[2] if len(sys.argv) > 2 else "Pong-v0"

episodes = 1500
env_name = "Pong-v0"

env = gym.make(env_name)

env = wrappers.Monitor(env, env_name, force=True)

agent = Agent(env.observation_space.shape, env.action_space.n)

for i_episodes in range(episodes):
    state = env.reset()
    totalreward = 0
    done = False
    state = RGBprocess(state)
    while not done:
        #env.render()
        action = agent.act(state)
        new_state, reward, done, info = env.step(action)
        new_state = RGBprocess(new_state)
        agent.remember(state, action, reward, new_state, done)
        state = new_state
        totalreward += reward
    agent.memory_replay(batch_size)
    if done:
        print("{} episode, score = {} ".format(i_episodes + 1, totalreward))
        agent.save_model()

env.close()
gym.upload(env_name, api_key='sk_WRCITkqmTJKYB9hvBk5tPA')

[2017-08-17 19:14:53,061] Making new env: Pong-v0
[2017-08-17 19:14:53,274] Clearing 14 monitor files from previous run (because force=True was provided)
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=8, activation="relu", padding="same", input_shape=(80, 80, 1..., strides=(4, 4))`
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=4, activation="relu", padding="same", strides=(2, 2))`
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=3, activation="relu", padding="same", strides=(1, 1))`
[2017-08-17 19:14:53,371] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Test/Pong-v0/openaigym.video.0.3656.v

1 0.0 [[-8.30639458  2.42786312 -0.16547194 -5.06692791 -8.32464886  7.39558268]]
2 0.0 [[ 134.43437195   23.33062935   67.22894287  115.56312561   83.20602417
    43.92967987]]
0 0.0 [[ 238.6413269   233.59402466  226.02151489  231.82173157  224.78735352
   176.68984985]]
4 0.0 [[ 513.4362793   576.06060791  472.87588501  515.45825195  533.48175049
   659.53143311]]
1 0.0 [[ 1062.51647949  1082.60632324   879.36749268  1038.57324219
   1114.20117188  1295.07531738]]
0 0.0 [[ 1994.8951416   2065.41967773  1682.58776855  1984.41760254
   2127.24121094  2255.18676758]]
0 0.0 [[ 3530.93481445  3707.9128418   3280.76269531  3525.12963867
   3794.83447266  3733.8996582 ]]
3 0.0 [[ 6280.28662109  6347.72802734  6424.16650391  6225.23730469
   6496.42333984  6284.13818359]]
2 0.0 [[ 11061.39746094  10829.97070312  11334.13183594  11090.11132812
   10887.18359375  10885.01464844]]
0 0.0 [[ 18003.27539062  17003.453125    17811.15429688  18073.09570312
   17240.15429688  17310.5859375 ]]
1 0.0 

[2017-08-17 19:14:56,548] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Test/Pong-v0/openaigym.video.0.3656.video000001.mp4


1 episode, score = -20.0 
3 0.0 [[ 289131.4375   281248.46875  282500.34375  287983.3125   283886.78125
   287498.90625]]
0 0.0 [[ 391579.03125  388205.03125  390775.875    388870.6875   387802.
   385099.8125 ]]
1 0.0 [[ 529806.5625   534563.5625   536518.9375   530393.875    533573.5
   523572.53125]]
3 0.0 [[ 706889.6875  717124.25    715300.6875  710862.8125  720075.625
   704325.5625]]
2 0.0 [[ 920483.4375  935544.125   927301.      928192.8125  940171.875
   926321.375 ]]
0 0.0 [[ 1184682.75   1204235.375  1183726.375  1197365.75   1201875.     1204349.75 ]]
0 0.0 [[ 1532133.25   1533467.75   1519722.875  1540076.25   1535228.375
   1550578.125]]
4 0.0 [[ 1957957.75   1922477.625  1923346.5    1958447.625  1932904.875
   1959959.   ]]
4 0.0 [[ 2466396.5   2418499.75  2433742.75  2452855.    2429197.25  2434967.5 ]]
1 0.0 [[ 3050892.75  3014517.    3046385.5   3038424.5   3020982.5   3000197.75]]
1 0.0 [[ 3764831.    3779332.25  3808277.75  3789610.5   3783659.    3738758.  ]]
0 0

3 0.0 [[  5.55014144e+09   5.55402035e+09   5.55366349e+09   5.55362355e+09
    5.54770176e+09   5.55403981e+09]]
5 0.0 [[  5.38927360e+09   5.39648768e+09   5.39291085e+09   5.39314381e+09
    5.38753485e+09   5.39175117e+09]]
3 0.0 [[  5.22664858e+09   5.22674381e+09   5.22501837e+09   5.22519552e+09
    5.22755430e+09   5.22440038e+09]]
5 0.0 [[  5.05054618e+09   5.04430746e+09   5.04610458e+09   5.04607232e+09
    5.05286349e+09   5.04672768e+09]]
5 0.0 [[  4.90310246e+09   4.89924250e+09   4.90203750e+09   4.90185318e+09
    4.90422272e+09   4.90282598e+09]]
4 -1.0 [[  4.79541146e+09   4.79844966e+09   4.79896013e+09   4.79879731e+09
    4.79424717e+09   4.79881216e+09]]
5 0.0 [[  4.66748877e+09   4.67311514e+09   4.67041382e+09   4.67053926e+09
    4.66569370e+09   4.66943181e+09]]
1 1.0 [[  4.55323034e+09   4.55484416e+09   4.55220480e+09   4.55246592e+09
    4.55256883e+09   4.55182029e+09]]
5 1.0 [[  4.38732186e+09   4.38413978e+09   4.38305690e+09   4.38328218e+09
    4.38871

...

0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 -1.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
1482 episode, score = -21.0 
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 -1.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ 

0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
1494 episode, score = -21.0 
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
1495 episode, score = -21.0 
0 0.0 [[ nan  nan  nan  nan  nan  nan]]
0 0.0 [[ nan  nan  nan

[2017-08-17 20:42:34,505] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Test/Pong-v0')
[2017-08-17 20:42:34,508] [Pong-v0] Uploading 1500 episodes of training data


0 0.0 [[ nan  nan  nan  nan  nan  nan]]
1500 episode, score = -21.0 


[2017-08-17 20:42:35,813] [Pong-v0] Uploading videos of 11 training episodes (485600 bytes)
[2017-08-17 20:42:39,681] [Pong-v0] Creating evaluation object from Pong-v0 with learning curve and training video
[2017-08-17 20:42:39,989] 
****************************************************
You successfully uploaded your evaluation on Pong-v0 to
OpenAI Gym! You can find it at:

    https://gym.openai.com/evaluations/eval_Zs4sltl3RqStxmsKGzZnaQ

****************************************************


In [ ]:
env.observation_space.shape

In [8]:
state.shape

(1, 80, 80, 1)

In [ ]:
reward

In [3]:
test = agent.model.predict(state)

In [4]:
test[0]

array([ nan,  nan,  nan,  nan,  nan,  nan], dtype=float32)

In [ ]:
env.reset()
env.close()

In [ ]:
len(agent.memory)

In [ ]:
processed_observation = Image.fromarray(state, 'RGB')
processed_observation = processed_observation.convert('L')
processed_observation = processed_observation.resize((80, 80))
processed_observation = np.array(processed_observation)
processed_observation = processed_observation.reshape(processed_observation.shape[0], processed_observation.shape[1], 1)

In [6]:
new_state

array([[[[ 87],
         [ 87],
         [ 87],
         ..., 
         [ 87],
         [ 87],
         [ 87]],

        [[ 87],
         [ 87],
         [ 87],
         ..., 
         [ 87],
         [ 87],
         [ 87]],

        [[ 87],
         [ 87],
         [ 87],
         ..., 
         [ 87],
         [ 87],
         [ 87]],

        ..., 
        [[236],
         [236],
         [236],
         ..., 
         [236],
         [236],
         [236]],

        [[236],
         [236],
         [236],
         ..., 
         [236],
         [236],
         [236]],

        [[236],
         [236],
         [236],
         ..., 
         [236],
         [236],
         [236]]]], dtype=uint8)

In [ ]:
def RGBprocess(new_state):
        processed_observation = Image.fromarray(new_state, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation

In [5]:
RGBprocess(state)

ValueError: Too many dimensions: 4 > 3.

In [ ]:
img.show()